# Логистическая регрессия с методом максимального правдоподобия
Этот ноутбук предназначен для Google Colab. В нем подробно объясняются все этапы работы с данными: их предобработка, разбиение на обучающую и тестовую выборки, а также реализация метода максимального правдоподобия для логистической регрессии.

## 1. Импорт библиотек
Сначала импортируем необходимые библиотеки:
- `numpy` для работы с массивами и линейной алгеброй
- `pandas` для работы с табличными данными
- `seaborn` и `matplotlib` для визуализации данных
- `scipy.optimize` для оптимизации функции правдоподобия
- `sklearn.model_selection` для разделения данных на обучающую и тестовую выборки

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
%matplotlib inline

## 2. Загрузка и анализ данных
Для демонстрации мы будем использовать набор данных о пассажирах Титаника. Этот набор данных содержит информацию о пассажирах, включая их возраст, пол, класс обслуживания и другие характеристики, которые могут повлиять на их вероятность выживания. Наша задача — предсказать, выжил пассажир или нет.

In [ ]:
url = 'https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/titanic/train.csv'
df = pd.read_csv(url)
df.head()

## 3. Очистка данных
Перед обучением модели необходимо подготовить данные:
- Преобразуем названия столбцов к нижнему регистру
- Заполним пропущенные значения в колонке `age` медианой
- Преобразуем категориальные переменные (пол, порт посадки) в числовой формат

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df['age'].fillna(df['age'].median(), inplace=True)
df['sex'] = (df['sex'] == 'male').astype(int)
df['embarked'] = df['embarked'].fillna('S')
df = pd.get_dummies(df, columns=['embarked'], drop_first=True)
df.dtypes

## 4. Разделение данных
Выбираем важные признаки для модели и разделяем данные на обучающую и тестовую выборки.

In [ ]:
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked_Q', 'embarked_S']
X = df[features]
y = df['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

## 5. Метод максимального правдоподобия
Функция логарифма правдоподобия вычисляет вероятность наблюдаемых данных при заданных параметрах модели. Мы будем минимизировать отрицательное правдоподобие, так как `scipy.optimize.minimize` выполняет минимизацию.

In [ ]:
def log_likelihood(theta, X, y):
    z = np.dot(X, theta)
    log_l = np.sum(y * z - np.log(1 + np.exp(z)))
    return -log_l

## 6. Обучение модели логистической регрессии
Мы используем метод BFGS для минимизации отрицательного логарифма правдоподобия и получения оптимальных коэффициентов модели.

In [ ]:
def train_mle(X, y):
    n_features = X.shape[1]
    initial_theta = np.zeros(n_features)
    result = minimize(log_likelihood, initial_theta, args=(X, y), method='BFGS')
    return result.x

# Добавляем единичный столбец для свободного члена (intercept)
X_train_mle = np.hstack((np.ones((X_train.shape[0], 1)), X_train))

# Обучаем модель
theta_opt = train_mle(X_train_mle, y_train)
theta_opt

## 7. Интерпретация коэффициентов
Каждый коэффициент показывает, как соответствующий признак влияет на вероятность выживания пассажира. Чем больше коэффициент, тем сильнее влияние.

In [ ]:
feature_names = ['Intercept'] + features
for name, coef in zip(feature_names, theta_opt):
    print(f'{name}: {coef:.4f}')